In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

import missingno as msno
from tqdm import tqdm_notebook

In [2]:
train_iden = pd.read_csv("train_identity.csv")
train_tran = pd.read_csv("train_transaction.csv")
test_iden = pd.read_csv("test_identity.csv")
test_tran = pd.read_csv("test_transaction.csv")

In [3]:
train_sample = train_tran[['isFraud',
                           'TransactionDT',	'TransactionAmt','card1',
                           'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 
       'C8', 'C9', 'C10', 'C11','C12', 'C13', 'C14','D1',
                           
       'V95','V96','V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
       'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
       'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123',
       'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132','V133', 'V134', 'V135', 'V136','V137',
                           
       'V280','V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(train_sample.mean())

In [4]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 
import imblearn.under_sampling as usam
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [7]:
X = sample.drop('isFraud', axis = 1)
Y = sample['isFraud']
feature_names = X.columns.tolist() # 컬럼을 리스트화
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
rus = usam.RandomUnderSampler()
undersampled_X, undersampled_Y = rus.fit_sample(Train_X, Train_Y)
model = GradientBoostingClassifier(n_estimators =  600, random_state  = 13,
                                   max_depth = 7, learning_rate = 0.2).fit(undersampled_X, undersampled_Y)
# max_depth가 높으면 이상하게 스코어가 낮아짐(제출시)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.8655097902155386


In [22]:
from sklearn.externals import joblib
joblib.dump(model, 'under_sampling_model.pkl')
model = joblib.load('under_sampling_model.pkl') 

In [23]:
test_sample = test_tran[[
                           'TransactionDT',	'TransactionAmt','card1',
                           'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 
       'C8', 'C9', 'C10', 'C11','C12', 'C13', 'C14','D1',
                           
       'V95','V96','V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
       'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
       'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123',
       'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132','V133', 'V134', 'V135', 'V136','V137',
                           
       'V280','V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = test_sample.fillna(train_sample.mean())

In [24]:
predicted = model.predict(sample)

In [25]:
dic = {'TransactionID':test_tran['TransactionID'],
       'isFraud':predicted}
submission = pd.DataFrame(dic)
len (submission)

506691

In [26]:
submission.to_csv('submission.csv', index = False)